In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from skimage.util import view_as_windows

from framework.pixelhop import *
from framework.utils import *

from skimage.measure import block_reduce
import xgboost as xgb
import warnings, gc
import time
import matplotlib.pyplot as plt

from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.__version__)

import os
print(os.getcwd())

from framework.dftloss import *

import lightgbm as lgb
import math
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix
import pandas as pd
import seaborn as sn


# ---------- Load MNIST data and split ----------
(x_train, y_train), (x_test,y_test) = cifar10.load_data()
# -----------Data Preprocessing-----------
x_train = np.asarray(x_train,dtype='float32')
x_test = np.asarray(x_test,dtype='float32')
y_train = np.asarray(y_train,dtype='int')
y_test = np.asarray(y_test,dtype='int')
print("training with input:", x_train.shape)


# Train pixelhop model

In [ ]:
# Parameter definition

SaabArgs = [
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':False, 'cw': False}, # PQR transform in this step
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            # {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'cw': True}
            ]

shrinkArgs = [
            # {'func':Shrink, 'win':1, 'pad':true/false, 'stride': 1},
            {'func':NoShrink, 'win':1, 'pad':0, 'stride': 1, 'pooling':0},#PQR
            {'func':Shrink, 'win':5, 'pad':0, 'stride': 1, 'pooling':1,'poolingParms' :(3,2,1)},#conv1
            {'func':Shrink, 'win':5, 'pad':0, 'stride': 1, 'pooling':1,'poolingParms' :(3,2,1)},#conv2
            {'func':Shrink, 'win':5, 'pad':0, 'stride': 1, 'pooling':1,'poolingParms' :(3,2,1)},#conv3
            # {'func':Shrink, 'win':3, 'pad':0, 'stride': 1, 'pooling':0,'poolingParms' :(3,2,1)},#conv4
            # {'func':Shrink, 'win':5, 'pad':0, 'stride': 1, 'pooling':0},#conv3
            ]

concatArg = {'func':Concat}



In [ ]:
# Model training:

threshold1 = 0.0001 # for splitting, th2 <= kernels <= th1 will be treated as leaf nodes
threshold2 = 0.00005 # for feature selections, discard these kernels directly after each saab training
depth = 4

t1 = time.time()

# mymodel = Pixelhop(depth=depth, TH1=threshold1, TH2=threshold2, \
#     SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg, DCAC= True)

mymodel = Pixelhop(depth=depth, TH1=threshold1, TH2=threshold2, \
    SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg, DCAC= False)
mymodel.fit(x_train)
print(time.time() - t1)

# Feature Extraction:

In [ ]:
t0 = time.time()

feattrain = mymodel.transform(x_train)[-1] #hop -1
feattrain = feattrain.reshape(feattrain.shape[0],-1)

feattest = mymodel.transform(x_test)[-1]  #hop -1
feattest = feattest.reshape(feattest.shape[0],-1)

print("extraction time:", time.time() - t0)

# DFT on pixelhop features:

In [ ]:
# evaluate features using DFT

from framework.dftloss import split_process_we

dfttestfeat = feattrain
# K_for_split = 256
K_for_split = 32 # try less than 128
num_classes = 10
nSamples = dfttestfeat.shape[0]

lossrecord = []

print(dfttestfeat.shape)
print(y.shape)

for featidx in tqdm(range(dfttestfeat.shape[-1])):
    
    X = dfttestfeat[:,featidx]
    X = X.astype('float64')

    splits = np.linspace(X.min(), X.max(), K_for_split)
    losstmp = []
    for i in range(1,splits.shape[0]-1):
        
        losstmp.append(split_process_we(X , y_train, splits[i], numclass = num_classes))

    lossrecord.append(np.min(losstmp))

plt.plot(sorted(lossrecord))
plt.show()


In [ ]:
# select top features for training:

featidx = np.argsort(lossrecord)
featlen = 500

feattrain_dft = feattrain[:, featidx[:featlen]]
feattest_dft = feattest[:, featidx[:featlen]]

# Train GBDT as classifier:

In [ ]:
from sklearn.model_selection import train_test_split

X_fit, X_val, y_fit, y_val = train_test_split(
    feattrain_dft, y_train, test_size=0.01, random_state=42)

print("train data:", X_fit.shape, y_fit.shape)
print("val data:", X_val.shape, y_val.shape)

In [ ]:
# fuse with 10 outputs as input features to train new xgboost


X_fit, X_val, y_fit, y_val = train_test_split(
        hop_predictions_train, Ytrain, test_size=0.2, random_state=42)
print("train data:", X_fit.shape, y_fit.shape)
print("val data:", X_val.shape, y_val.shape)


fit = lgb.Dataset(
    X_fit, y_fit,
)

val = lgb.Dataset(
    X_val, y_val,
)

maxdepth = 1
lr0 = 0.01 # make sure GD at the begining
lr_final = 0.001 # make sure final converge faster
k = -0.001 # lower -> more smooth
num_rounds = 500
rounds = np.arange(num_rounds)
rounds_lr = lr0 * math.e**(k * rounds)
evals_result = {} 

t0 = time.time()
model = lgb.train(
    params={
        'learning_rate': 0.1,
        'max_depth': maxdepth,
        # 'num_leaves': 10,
        'device' : 'gpu',
        # 'gpu_platform_id' : 0,
        # 'gpu_device_id' : 0 ,
        # 'num_gpu' : 2,
        'objective': 'multiclass',
        'num_class':10,
        # 'feature_fraction': 0.4,
        # 'bagging_fraction': 0.6,
        # 'bagging_freq': 10,
        'verbose' : -1
           },
    train_set=fit,
    num_boost_round=num_rounds,
    valid_sets=(fit, val),
    valid_names=('fit', 'val'),
    # early_stopping_rounds=200,
    evals_result=evals_result,
    verbose_eval=100,
    # fobj=fl.lgb_obj,
    # feval=fl.lgb_eval,
    callbacks=[lgb.reset_parameter(learning_rate=lambda \
        current_round: max(lr0 * math.e**(k * current_round), lr_final))]
)

y_preds =model.predict(X_fit)
y_train_preds = []
for x in y_preds:
    y_train_preds.append(np.argmax(x))
y_train_preds = np.array(y_train_preds)
train_accuracy = accuracy_score(y_fit, y_train_preds)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))

y_test_preds = []
y_preds =model.predict(hop_predictions_test)
for x in y_preds:
    y_test_preds.append(np.argmax(x))
y_test_preds = np.array(y_test_preds)

pred_accuracy_score = accuracy_score(y_test, y_test_preds)
# pred_recall_score = recall_score(y_test, y_test_pred, average='macro')
print("Test Accuary: %.2f%%" % (pred_accuracy_score* 100.0))

# Inference on Test

In [ ]:
t0 = time.time()
y_test_preds = []
y_preds =model.predict(feattest_dft)
for x in y_preds:
    y_test_preds.append(np.argmax(x))
y_test_preds = np.array(y_test_preds)

pred_accuracy_score = accuracy_score(y_test, y_test_preds)
print("Test Accuary: %.2f%%" % (pred_accuracy_score* 100.0))

cnf_matrix = confusion_matrix(y_test, y_test_preds)
df_cm = pd.DataFrame(cnf_matrix)
sn.heatmap(df_cm)
plt.show()